# Setup

In [1]:
EXP_NAME='Santander Release Pipeline'
OUTPUT_DIR='gs://kubeflow-pipelines-demo/tfx'

PROJECT_NAME = 'kf-pipelines'

# Create a release experiment in the Kubeflow pipeline

Kubeflow Pipeline requires having an Experiment before making a run. An experiment is a group of comparable runs

In [2]:
import kfp
from kfp import compiler
import kfp.dsl as dsl

import kfp.notebook
import kfp.gcp as gcp

client = kfp.Client()
exp = client.create_experiment(name=EXP_NAME)

# Define the release pipeline

In [3]:
import kfp
from kfp import components
from kfp import dsl
from kfp import gcp
from kfp import onprem

platform = 'GCP'

kubeflow_deploy_op = components.load_component_from_file('../pipeline_steps/serving/deployer/component.yaml')

In [4]:
@dsl.pipeline(
    name='Santander Customer Transaction Prediction Release Pipeline',
    description='Example pipeline that releases the trained classification model for Santander customer transaction.'
)
def santander_transaction_classification(
        output,
        project,
):
    tf_server_name = 'kfdemo-service'

    if platform != 'GCP':
        vop = dsl.VolumeOp(
            name="create_pvc",
            resource_name="pipeline-pvc",
            modes=dsl.VOLUME_MODE_RWM,
            size="1Gi"
        )

        checkout = dsl.ContainerOp(
            name="checkout",
            image="alpine/git:latest",
            command=["git", "clone", "https://github.com/kubeflow/pipelines.git", str(output) + "/pipelines"],
        ).apply(onprem.mount_pvc(vop.outputs["name"], 'local-storage', output))
        checkout.after(vop)

    if platform == 'GCP':
        deploy = kubeflow_deploy_op(
            model_dir=str(
                'gs://kubeflow-pipelines-demo/tfx/0b22081a-ed94-11e9-81fb-42010a800160/santander-customer-transaction-prediction-95qxr-268134926/data') + '/export/export',
            server_name=tf_server_name
        )
    else:
        deploy = kubeflow_deploy_op(
            cluster_name=project,
            model_dir=str(
                'gs://kubeflow-pipelines-demo/tfx/0b22081a-ed94-11e9-81fb-42010a800160/santander-customer-transaction-prediction-95qxr-268134926/data') + '/export/export',
            pvc_name=vop.outputs["name"],
            server_name=tf_server_name
        )

    webapp = dsl.ContainerOp(
        name='webapp',
        image='us.gcr.io/kf-pipelines/ml-pipeline-webapp-launcher:v0.3',
        arguments=["--model_name", 'santanderapp']

    )
    webapp.after(deploy)

    steps = [deploy, webapp]

    for step in steps:
        if platform == 'GCP':
            step.apply(gcp.use_gcp_secret('user-gcp-sa'))
        else:
            step.apply(onprem.mount_pvc(vop.outputs["name"], 'local-storage', output))

# Compile the pipeline

Compile the pipeline into a tar package

In [5]:
compiler.Compiler().compile(santander_transaction_classification, 'santander_release_pipeline.zip')

# Submit the run with parameters

In [6]:
import datetime
import time

run = client.run_pipeline(exp.id, 'santander_release_pipeline-' + time.strftime("%Y%m%d-%H%M%S"), 'santander_release_pipeline.zip', params={'output':'gs://kubeflow-pipelines-demo/tfx','project':'kf-pipelines'})

# Access the frontend

In [9]:
!cd .. && make frontend-external-ip

kubectl get service santanderapp-webappsvc
NAME                     TYPE           CLUSTER-IP     EXTERNAL-IP     PORT(S)        AGE
santanderapp-webappsvc   LoadBalancer   10.11.247.81   34.67.233.115   80:31689/TCP   45s


# Resources
## - Git repo: https://github.com/Mastercard/Mastercard-Labs-ML-Pipeline
